# Logic Networks

In [76]:
#imports
from vpython import *
import numpy as np
import math
# importing matplotlib module
from matplotlib import pyplot as plt

In [77]:
#create generic gate class
class TwoGate:
    def __init__(self, a, b, out, logic = "nand"):
        self.a = a
        self.b = b
        self.out = out
        
        self.logic = logic
        #logic determines what kind of gate
        
        a.addConnect(self)
        b.addConnect(self)
        
    def __str__(self):
        return "Gate Type: " + self.logic + "; Input Nets: " + str(self.a) +", " + str(self.b) + "; Output Nets: " + str(self.out)

    def readOut(self):
        print("Output of this gate (a,b,out): ")
        print(self.a)
        print(self.b)
        print(self.out)
        
    def doNand(self):
        if self.a.getVal() == 1 and self.b.getVal() == 1:
            self.out.setVal(0)
        else:
            self.out.setVal(1)
            
    def doNor(self):
        if self.a.val == 0 and self.b.val == 0:
            self.out.val = 1
        else:
            self.out.val = 0
            
    def doLogic(self):
        if(self.logic == "nand"):
            self.doNand()
        elif(self.logic == "nor"):
            self.doNor()
            
    def update(self):
        pass
#
#create net class for connections
class Net():
    def __init__(self,val=-1):
        self.val = val
        self.connections = []
        
    def __str__(self):
        return "Val: " + str(self.val)
        
    def probe(self):
        print(self.val)
        
    def getVal(self):
        return self.val
    
    def setVal(self,val):
        self.val = val
        for cnx in self.connections:
            cnx.update()
    
    def flipVal(self):
        self.val = !self.val
        for cnx in self.connections:
            cnx.update()
    
    def addConnect(self,connection):
        self.connections.append(connection)
        
    def rmvConnect(self,connection):
        self.connections.remove(connection)

In [78]:
#manual test of gate class with string parameter for logic type
netHigh = Net(1)
netLow = Net(0)
netOut = Net(0)

testNand0 = TwoGate(netLow,netLow,netOut,"nand")
testNand0.doLogic()
testNand0.readOut()

testNand1 = TwoGate(netLow,netHigh,netOut,"nand")
testNand1.doLogic()
testNand1.readOut()

testNand2 = TwoGate(netHigh,netLow,netOut,"nand")
testNand2.doLogic()
testNand2.readOut()

testNand3 = TwoGate(netHigh,netHigh,netOut,"nand")
testNand3.doLogic()
testNand3.readOut()


testNor0 = TwoGate(netLow,netLow,netOut,"nor")
testNor0.doLogic()
testNor0.readOut()

testNor1 = TwoGate(netLow,netHigh,netOut,"nor")
testNor1.doLogic()
testNor1.readOut()

testNor2 = TwoGate(netHigh,netLow,netOut,"nor")
testNor2.doLogic()
testNor2.readOut()

testNor3 = TwoGate(netHigh,netHigh,netOut,"nor")
testNor3.doLogic()
testNor3.readOut()


Output of this gate (a,b,out): 
Val: 0
Val: 0
Val: 1
Output of this gate (a,b,out): 
Val: 0
Val: 1
Val: 1
Output of this gate (a,b,out): 
Val: 1
Val: 0
Val: 1
Output of this gate (a,b,out): 
Val: 1
Val: 1
Val: 0
Output of this gate (a,b,out): 
Val: 0
Val: 0
Val: 1
Output of this gate (a,b,out): 
Val: 0
Val: 1
Val: 0
Output of this gate (a,b,out): 
Val: 1
Val: 0
Val: 0
Output of this gate (a,b,out): 
Val: 1
Val: 1
Val: 0


In [81]:
#connect gates with netlist
netA = Net(0)
netB = Net(0)
netC = Net(0)
netD = Net(0)

nandGate1 = TwoGate(netA,netB,netC,"nand")
norGate1 = TwoGate(netA,netB,netD,"nor")

values = graph(title="Nand Gate thru time",xtitle="timestep",ytitle="value",fast=False,width=800)
f1 = gcurve(color = color.cyan)
f2 = gcurve(color = color.red)
f3 = gcurve(color = color.green)

for i in range(4):
    rate(100)
    if(i == 0):
        netA.setVal(0)
        netB.setVal(0)
    elif(i == 1):
        netA.setVal(1)
        netB.setVal(0)
    elif(i == 2):
        netA.setVal(0)
        netB.setVal(1)
    elif(i == 3):
        netA.setVal(1)
        netB.setVal(1)
    
    nandGate1.doLogic()
    norGate1.doLogic()

    print("inputs")
    netA.probe()
    netB.probe()
    print("nand output")
    netC.probe()
    print("nor output")
    netD.probe()
    print("\n")
    
    

inputs
0
0
nand output
1
nor output
1


inputs
1
0
nand output
1
nor output
0


inputs
0
1
nand output
1
nor output
0


inputs
1
1
nand output
0
nor output
0




# Results of Independent Gates
Since I could not get Vpython graphs to show up, I have manually printed the results for the combinations of 00,01,10,11 for both nand and nor gates. The outputs of each, depending on the same inputs at the same times, match the truth tables of their respective logical operations.

In [ ]:
#Redefining classes with time for priority queue
from queue import PriorityQueue

#create generic gate class
class TwoGate:
    def __init__(self, a, b, out, logic = "nand", delay = 5):
        self.a = a
        self.b = b
        self.out = out
        #logic determines what kind of gate
        self.logic = logic
        #delay factor for timing
        self.delay = delay
        
        a.addConnect(self)
        b.addConnect(self)
        
    def __str__(self):
        return "Gate Type: " + self.logic + "; Input Nets: " + str(self.a) +", " + str(self.b) + "; Output Nets: " + str(self.out)

    def readOut(self):
        print("Output of this gate (a,b,out): ")
        print(self.a)
        print(self.b)
        print(self.out)
        
    def doNand(self):
        if self.a.getVal() == 1 and self.b.getVal() == 1:
            self.out.setVal(0)
        else:
            self.out.setVal(1)
            
    def doNor(self):
        if self.a.val == 0 and self.b.val == 0:
            self.out.val = 1
        else:
            self.out.val = 0
            
    def doLogic(self):
        if(self.logic == "nand"):
            self.doNand()
        elif(self.logic == "nor"):
            self.doNor()
            
    def update(self):
        pass
    
    def getDelay(self):
        return self.delay
#
#create net class for connections
class Net():
    def __init__(self,val=-1):
        self.val = val
        self.connections = []
        
    def __str__(self):
        return "Val: " + str(self.val)
        
    def probe(self):
        print(self.val)
        
    def getVal(self):
        return self.val
    
    def setVal(self,val):
        self.val = val
        for cnx in self.connections:
            cnx.update()
    
    def flipVal(self):
        self.val = !self.val
        for cnx in self.connections:
            cnx.update()
    
    def addConnect(self,connection):
        self.connections.append(connection)
        
    def rmvConnect(self,connection):
        self.connections.remove(connection)

In [72]:
#create events class, priority queue
#includes net location, value, time of event

In [ ]:
#system test of combinational logic
# -at least 3 gates of at least 2 inputs
# -at least 2 levels

In [ ]:
#NAND gate D-latch, sequential logic